In [1]:
from datasets import load_dataset

# Укажите тот же путь, который вы использовали при скачивании
cache_dir = "./my_dataset_folder"

# Библиотека проверит папку и мгновенно загрузит данные из кэша
dataset = load_dataset("deepcopy/MathWriting-human", cache_dir=cache_dir)

sample = dataset["train"][5]
image = sample["image"]
latex = sample["latex"]

In [2]:
txt = []

In [3]:
from tqdm import trange

for i in trange(len(dataset["train"])):
    cur_text = list(map(int, dataset["train"][i]["latex"].encode("utf-8")))
    txt.append(cur_text)

100%|██████████| 229864/229864 [00:58<00:00, 3897.60it/s]


In [4]:

print(txt[6])

[40, 92, 98, 101, 103, 105, 110, 123, 109, 97, 116, 114, 105, 120, 125, 45, 49, 38, 45, 49, 92, 92, 32, 49, 38, 48, 92, 101, 110, 100, 123, 109, 97, 116, 114, 105, 120, 125, 41]


In [5]:
vocab_size = 300
num_merges = vocab_size - 256

In [6]:
def get_stats(x): # x - list of labels
    counts = {}
    for label in x:
        for ft, sec in zip(label, label[1:]):
            counts[(ft, sec)] = counts.get((ft, sec), 0) + 1
    return counts

In [7]:
counts = get_stats(txt)

In [8]:

counts[(94, 123)]

156749

In [9]:
max(counts, key=counts.get)

(94, 123)

In [10]:
merges = {} #(int, int) -> int

txt_copy = txt.copy() # list of labels for each example

def merge(idx, focus, txt_copy):
    first, second = focus
    repl_txt = []
    for label in txt_copy:
        temp = []
        i = 0
        while i < len(label):
            if i + 1 < len(label) and label[i] == first and label[i + 1] == second:
                temp.append(idx)
                i += 2
            else:
                temp.append(label[i])
                i += 1
        repl_txt.append(temp)
    return repl_txt


for i in range(num_merges):
    # we need to count pair with the highest frequency
    counts = get_stats(txt_copy)
    focus = max(counts, key=counts.get)
    idx = 256 + i
    print(f"merge {focus} to {idx}")
    txt_copy = merge(idx, focus, txt_copy)
    merges[focus] = idx


merge (94, 123) to 256
merge (95, 123) to 257
merge (125, 125) to 258
merge (114, 97) to 259
merge (99, 123) to 260
merge (92, 102) to 261
merge (261, 259) to 262
merge (262, 260) to 263
merge (125, 123) to 264
merge (105, 110) to 265
merge (114, 116) to 266
merge (97, 116) to 267
merge (120, 125) to 268
merge (114, 105) to 269
merge (92, 115) to 270
merge (92, 112) to 271
merge (109, 267) to 272
merge (123, 272) to 273
merge (273, 269) to 274
merge (274, 268) to 275
merge (113, 266) to 276
merge (270, 276) to 277
merge (256, 50) to 278
merge (277, 123) to 279
merge (258, 123) to 280
merge (125, 61) to 281
merge (92, 101) to 282
merge (101, 103) to 283
merge (97, 108) to 284
merge (92, 98) to 285
merge (116, 97) to 286
merge (111, 116) to 287
merge (110, 100) to 288
merge (125, 256) to 289
merge (92, 265) to 290
merge (125, 41) to 291
merge (92, 99) to 292
merge (92, 92) to 293
merge (293, 32) to 294
merge (101, 123) to 295
merge (100, 287) to 296
merge (257, 105) to 297
merge (285, 28

In [11]:
merges

{(94, 123): 256,
 (95, 123): 257,
 (125, 125): 258,
 (114, 97): 259,
 (99, 123): 260,
 (92, 102): 261,
 (261, 259): 262,
 (262, 260): 263,
 (125, 123): 264,
 (105, 110): 265,
 (114, 116): 266,
 (97, 116): 267,
 (120, 125): 268,
 (114, 105): 269,
 (92, 115): 270,
 (92, 112): 271,
 (109, 267): 272,
 (123, 272): 273,
 (273, 269): 274,
 (274, 268): 275,
 (113, 266): 276,
 (270, 276): 277,
 (256, 50): 278,
 (277, 123): 279,
 (258, 123): 280,
 (125, 61): 281,
 (92, 101): 282,
 (101, 103): 283,
 (97, 108): 284,
 (92, 98): 285,
 (116, 97): 286,
 (111, 116): 287,
 (110, 100): 288,
 (125, 256): 289,
 (92, 265): 290,
 (125, 41): 291,
 (92, 99): 292,
 (92, 92): 293,
 (293, 32): 294,
 (101, 123): 295,
 (100, 287): 296,
 (257, 105): 297,
 (285, 283): 298,
 (298, 265): 299}

In [57]:
stat1 = []
for label in txt:
    stat1.append(len(label))

stat2 = []
for label in txt_copy:
    stat2.append(len(label))

mean1 = sum(stat1) / len(stat1)
mean2 = sum(stat2) / len(stat2)
print(f"mean len of sequence before tokenization {mean1}")
print(f"mean len of sequence after tokenization {mean2}")
print(f"compression ratio: {mean1 / mean2}")

mean len of sequence before tokenization 27.792207566213065
mean len of sequence after tokenization 17.105022970104063
compression ratio: 1.6247980265672795


In [69]:
vocab = {merges[pair] : pair for pair in merges}

In [70]:
vocab

{256: (94, 123),
 257: (95, 123),
 258: (125, 125),
 259: (114, 97),
 260: (99, 123),
 261: (92, 102),
 262: (261, 259),
 263: (262, 260),
 264: (125, 123),
 265: (105, 110),
 266: (114, 116),
 267: (97, 116),
 268: (120, 125),
 269: (114, 105),
 270: (92, 115),
 271: (92, 112),
 272: (109, 267),
 273: (123, 272),
 274: (273, 269),
 275: (274, 268),
 276: (113, 266),
 277: (270, 276),
 278: (256, 50),
 279: (277, 123),
 280: (258, 123),
 281: (125, 61),
 282: (92, 101),
 283: (101, 103),
 284: (97, 108),
 285: (92, 98),
 286: (116, 97),
 287: (111, 116),
 288: (110, 100),
 289: (125, 256),
 290: (92, 265),
 291: (125, 41),
 292: (92, 99),
 293: (92, 92),
 294: (293, 32),
 295: (101, 123),
 296: (100, 287),
 297: (257, 105),
 298: (285, 283),
 299: (298, 265)}

In [79]:
merges

{(94, 123): 256,
 (95, 123): 257,
 (125, 125): 258,
 (114, 97): 259,
 (99, 123): 260,
 (92, 102): 261,
 (261, 259): 262,
 (262, 260): 263,
 (125, 123): 264,
 (105, 110): 265,
 (114, 116): 266,
 (97, 116): 267,
 (120, 125): 268,
 (114, 105): 269,
 (92, 115): 270,
 (92, 112): 271,
 (109, 267): 272,
 (123, 272): 273,
 (273, 269): 274,
 (274, 268): 275,
 (113, 266): 276,
 (270, 276): 277,
 (256, 50): 278,
 (277, 123): 279,
 (258, 123): 280,
 (125, 61): 281,
 (92, 101): 282,
 (101, 103): 283,
 (97, 108): 284,
 (92, 98): 285,
 (116, 97): 286,
 (111, 116): 287,
 (110, 100): 288,
 (125, 256): 289,
 (92, 265): 290,
 (125, 41): 291,
 (92, 99): 292,
 (92, 92): 293,
 (293, 32): 294,
 (101, 123): 295,
 (100, 287): 296,
 (257, 105): 297,
 (285, 283): 298,
 (298, 265): 299}

In [80]:
list([1])

[1]

In [90]:
def smart_pair_decoder(merges):
    smart_vocab = {}
    for pair in merges:
        if all(x < 256 for x in pair):
            smart_vocab[merges[pair]] = list(pair)
        else:
            a = smart_vocab.get(pair[0], [pair[0]])
            b = smart_vocab.get(pair[1], [pair[1]])
            new = list(a) + list(b)
            smart_vocab[merges[pair]] = new
    return smart_vocab


dec_merges = smart_pair_decoder(merges)


In [91]:
dec_merges

{256: [94, 123],
 257: [95, 123],
 258: [125, 125],
 259: [114, 97],
 260: [99, 123],
 261: [92, 102],
 262: [92, 102, 114, 97],
 263: [92, 102, 114, 97, 99, 123],
 264: [125, 123],
 265: [105, 110],
 266: [114, 116],
 267: [97, 116],
 268: [120, 125],
 269: [114, 105],
 270: [92, 115],
 271: [92, 112],
 272: [109, 97, 116],
 273: [123, 109, 97, 116],
 274: [123, 109, 97, 116, 114, 105],
 275: [123, 109, 97, 116, 114, 105, 120, 125],
 276: [113, 114, 116],
 277: [92, 115, 113, 114, 116],
 278: [94, 123, 50],
 279: [92, 115, 113, 114, 116, 123],
 280: [125, 125, 123],
 281: [125, 61],
 282: [92, 101],
 283: [101, 103],
 284: [97, 108],
 285: [92, 98],
 286: [116, 97],
 287: [111, 116],
 288: [110, 100],
 289: [125, 94, 123],
 290: [92, 105, 110],
 291: [125, 41],
 292: [92, 99],
 293: [92, 92],
 294: [92, 92, 32],
 295: [101, 123],
 296: [100, 111, 116],
 297: [95, 123, 105],
 298: [92, 98, 101, 103],
 299: [92, 98, 101, 103, 105, 110]}

In [95]:
for pair in dec_merges:
    lst = [bytes([x]) for x in dec_merges[pair]]

    tokens = b"".join(lst)
    text = tokens.decode("utf-8")
    print(text)


^{
_{
}}
ra
c{
\f
\fra
\frac{
}{
in
rt
at
x}
ri
\s
\p
mat
{mat
{matri
{matrix}
qrt
\sqrt
^{2
\sqrt{
}}{
}=
\e
eg
al
\b
ta
ot
nd
}^{
\in
})
\c
\\
\\ 
e{
dot
_{i
\beg
\begin


### save tokenyzer

In [13]:
merges

{(94, 123): 256,
 (95, 123): 257,
 (125, 125): 258,
 (114, 97): 259,
 (99, 123): 260,
 (92, 102): 261,
 (261, 259): 262,
 (262, 260): 263,
 (125, 123): 264,
 (105, 110): 265,
 (114, 116): 266,
 (97, 116): 267,
 (120, 125): 268,
 (114, 105): 269,
 (92, 115): 270,
 (92, 112): 271,
 (109, 267): 272,
 (123, 272): 273,
 (273, 269): 274,
 (274, 268): 275,
 (113, 266): 276,
 (270, 276): 277,
 (256, 50): 278,
 (277, 123): 279,
 (258, 123): 280,
 (125, 61): 281,
 (92, 101): 282,
 (101, 103): 283,
 (97, 108): 284,
 (92, 98): 285,
 (116, 97): 286,
 (111, 116): 287,
 (110, 100): 288,
 (125, 256): 289,
 (92, 265): 290,
 (125, 41): 291,
 (92, 99): 292,
 (92, 92): 293,
 (293, 32): 294,
 (101, 123): 295,
 (100, 287): 296,
 (257, 105): 297,
 (285, 283): 298,
 (298, 265): 299}

In [14]:
vocab = {idx: bytes([idx]) for idx in range(256)}

In [15]:
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [18]:
merges.items()

dict_items([((94, 123), 256), ((95, 123), 257), ((125, 125), 258), ((114, 97), 259), ((99, 123), 260), ((92, 102), 261), ((261, 259), 262), ((262, 260), 263), ((125, 123), 264), ((105, 110), 265), ((114, 116), 266), ((97, 116), 267), ((120, 125), 268), ((114, 105), 269), ((92, 115), 270), ((92, 112), 271), ((109, 267), 272), ((123, 272), 273), ((273, 269), 274), ((274, 268), 275), ((113, 266), 276), ((270, 276), 277), ((256, 50), 278), ((277, 123), 279), ((258, 123), 280), ((125, 61), 281), ((92, 101), 282), ((101, 103), 283), ((97, 108), 284), ((92, 98), 285), ((116, 97), 286), ((111, 116), 287), ((110, 100), 288), ((125, 256), 289), ((92, 265), 290), ((125, 41), 291), ((92, 99), 292), ((92, 92), 293), ((293, 32), 294), ((101, 123), 295), ((100, 287), 296), ((257, 105), 297), ((285, 283), 298), ((298, 265), 299)])

In [20]:
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

In [21]:
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [22]:
import pickle

# Сохранение
with open('tokenyzer/idx_to_byte.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [26]:
with open('tokenyzer/byte_to_idx.pkl', 'wb') as f:
    pickle.dump(merges, f)

In [30]:
def decode(ids):
    # ids - list of integers [1, 265, 3]
    tokens = b"".join(vocab[x] for x in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text


In [31]:
merges

{(94, 123): 256,
 (95, 123): 257,
 (125, 125): 258,
 (114, 97): 259,
 (99, 123): 260,
 (92, 102): 261,
 (261, 259): 262,
 (262, 260): 263,
 (125, 123): 264,
 (105, 110): 265,
 (114, 116): 266,
 (97, 116): 267,
 (120, 125): 268,
 (114, 105): 269,
 (92, 115): 270,
 (92, 112): 271,
 (109, 267): 272,
 (123, 272): 273,
 (273, 269): 274,
 (274, 268): 275,
 (113, 266): 276,
 (270, 276): 277,
 (256, 50): 278,
 (277, 123): 279,
 (258, 123): 280,
 (125, 61): 281,
 (92, 101): 282,
 (101, 103): 283,
 (97, 108): 284,
 (92, 98): 285,
 (116, 97): 286,
 (111, 116): 287,
 (110, 100): 288,
 (125, 256): 289,
 (92, 265): 290,
 (125, 41): 291,
 (92, 99): 292,
 (92, 92): 293,
 (293, 32): 294,
 (101, 123): 295,
 (100, 287): 296,
 (257, 105): 297,
 (285, 283): 298,
 (298, 265): 299}

In [32]:
def encode(text):
    tokens = list(text.encode("utf-8")) #[5, 167, 258]
    while len(tokens) >= 2:
        # run and count all consecutive pairs in bt
        stats = []
        for i, j in zip(tokens, tokens[1:]):
            stats.append((i, j))
        # next, we find pair with minimum idx in merges, to merge from up to down
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break # nothing else can be merged
        idx = merges[pair]
        tokens = merge(idx, pair, tokens)
    return tokens



In [33]:
encode("k/")

[107, 47]

In [34]:
decode([107, 47])

'k/'

In [35]:
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'